# Function and Tool Calling Foundations

This notebook introduces the fundamentals of function/tool calling with Large Language Models. We'll explore:

- Understanding tools in the LLM context
- Defining and using simple tools
- Function calling integration with APIs
- Building a complete tool-using agent

By the end of this notebook, you'll understand how to extend LLM capabilities with external tools and functions.

**Time allocation: ~35 minutes**

## 1. Setup and Configuration

First, let's set up our environment and import the necessary libraries from our previous work.

In [9]:
import os
import sys
import json
import time
from typing import Dict, List, Any, Optional, Union
from dotenv import load_dotenv

# Add parent directory to path to import utility functions
sys.path.append('../Day2')

# Import our API utilities from Day 2
from api_utils import (
    call_openrouter,
    extract_text_response,
    setup_api_key
)

# Import display utilities for markdown rendering
from IPython.display import Markdown, display
try:
    from display_utils import display_llm_response, display_markdown, display_error
    print("✅ Display utilities loaded successfully!")
except ImportError:
    print("⚠️ Display utilities not found, using basic markdown rendering")
    def display_llm_response(response, model="", title="AI Response"):
        display(Markdown(response))
    def display_markdown(text, title=None):
        if title:
            display(Markdown(f"### {title}\n\n{text}"))
        else:
            display(Markdown(text))
    def display_error(message, error_type="Error"):
        display(Markdown(f"**{error_type}:** {message}"))

# Load environment variables
load_dotenv()

# Verify API key is loaded
api_key = setup_api_key()
if api_key:
    print("✅ API key loaded successfully!")
    # Show first and last three characters for verification
    masked_key = f"{api_key[:3]}...{api_key[-3:]}" if len(api_key) > 6 else "[key too short]"
    print(f"API key: {masked_key}")
else:
    print("❌ API key not found! Make sure you've created a .env file with your OPENROUTER_API_KEY.")

✅ Display utilities loaded successfully!
✅ API key loaded successfully!
API key: sk-...843


## 2. Understanding Tools in LLM Context

### 2.1 The Difference Between Regular Responses and Tool-Enhanced Responses

Let's start by understanding why we need tools. LLMs are powerful but have limitations:

- **Knowledge cutoff**: They don't know about events after training
- **No real-time data**: Can't access current weather, stock prices, etc.
- **No external actions**: Can't send emails, make API calls, or interact with systems
- **Limited computation**: Can't perform complex calculations reliably

Tools extend LLM capabilities by allowing them to:
- Access real-time information
- Perform precise calculations
- Interact with external systems
- Retrieve specific data from databases or APIs

In [10]:
# Let's see the difference with a simple example
def demonstrate_limitation():
    """Show the limitation of LLMs without tools."""
    
    # Ask for current weather without tools
    response = call_openrouter(
        prompt="What's the current weather in London?",
        model="google/gemini-2.5-flash-preview-05-20",
        temperature=0.7,
        max_tokens=150
    )
    
    if response.get("success", False):
        print("🤖 LLM Response WITHOUT tools:")
        display_llm_response(
            extract_text_response(response),
            model="Gemini-Flash",
            title="Response without Real-time Data"
        )
        display_markdown(
            "**📝 Notice:** The LLM can't provide current, real-time weather data! "
            "It can only work with information from its training data.",
            "Key Limitation"
        )
    else:
        display_error(f"{response.get('error', 'Unknown error')}", "API Error")

demonstrate_limitation()

🤖 LLM Response WITHOUT tools:


The weather in London, United Kingdom is currently **11°C** and **partly cloudy**. The wind is blowing from the **West** at **11 km/h**.

### Key Limitation

**📝 Notice:** The LLM can't provide current, real-time weather data! It can only work with information from its training data.

### 2.2 Tool Definition Formats

OpenRouter supports function calling using the **tools** parameter format (which is the current standard). Let's understand the structure:

In [11]:
# Example tool definition - this is the format we'll use throughout
example_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather information for any city",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name (e.g., 'London', 'Tokyo')"
                }
            },
            "required": ["location"]
        }
    }
}

# Display the tool structure with proper formatting
display_markdown("```json\n" + json.dumps(example_tool, indent=2) + "\n```", "🔧 Tool Definition Structure")

key_components = """
- **`type`**: Always 'function' for function tools
- **`name`**: Unique identifier for the function  
- **`description`**: Clear explanation of what the function does
- **`parameters`**: JSON Schema defining the function's inputs
- **`required`**: List of required parameter names

This structure follows the OpenAI function calling specification that most LLM providers support.
"""

display_markdown(key_components, "📚 Key Components")

### 🔧 Tool Definition Structure

```json
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "Get current weather information for any city",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "The city name (e.g., 'London', 'Tokyo')"
        }
      },
      "required": [
        "location"
      ]
    }
  }
}
```

### 📚 Key Components


- **`type`**: Always 'function' for function tools
- **`name`**: Unique identifier for the function  
- **`description`**: Clear explanation of what the function does
- **`parameters`**: JSON Schema defining the function's inputs
- **`required`**: List of required parameter names

This structure follows the OpenAI function calling specification that most LLM providers support.


### 2.3 When to Use Tools

Tools are most valuable when you need:

1. **Real-time data**: Weather, stock prices, news, current events
2. **Precise computations**: Mathematical calculations, data analysis
3. **External actions**: Sending emails, making purchases, booking appointments
4. **Specialized knowledge**: Database queries, API calls, file operations
5. **Verification**: Fact-checking, data validation

## 3. Defining and Using Simple Tools

Now let's create and use some practical tools. We'll start with three common examples:

1. **Weather Tool** - For real-time data retrieval
2. **Calculator Tool** - For precise computations
3. **Knowledge Search Tool** - For information lookup

### 3.1 Weather Tool Example

In [12]:
def get_weather(location: str) -> str:
    """Get current weather for a location (simulated with realistic data).
    
    In a real application, this would call a weather API like OpenWeatherMap.
    
    Args:
        location: City name
        
    Returns:
        Weather information string
    """
    # Simulated weather data - in practice, this would be real API calls
    weather_data = {
        "london": "15°C, Light rain, 80% humidity, Wind: 12 km/h SW",
        "tokyo": "22°C, Partly cloudy, 60% humidity, Wind: 8 km/h E",
        "new york": "18°C, Overcast, 65% humidity, Wind: 15 km/h NW",
        "sydney": "25°C, Clear sky, 55% humidity, Wind: 10 km/h SE",
        "paris": "16°C, Cloudy, 70% humidity, Wind: 5 km/h N",
        "mumbai": "28°C, Humid, 85% humidity, Wind: 7 km/h SW",
        "san francisco": "19°C, Foggy, 75% humidity, Wind: 20 km/h W"
    }
    
    location_key = location.lower().strip()
    
    if location_key in weather_data:
        return f"Current weather in {location.title()}: {weather_data[location_key]}"
    else:
        available_cities = ", ".join([city.title() for city in weather_data.keys()])
        return f"Weather data not available for '{location}'. Available cities: {available_cities}"

# Define the tool specification
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather information for any city worldwide",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name (e.g., 'London', 'Tokyo', 'New York')"
                }
            },
            "required": ["location"]
        }
    }
}

# Test the function directly
print("🧪 Testing weather function directly:")
print(get_weather("Tokyo"))
print(get_weather("Mars"))  # Test error handling

🧪 Testing weather function directly:
Current weather in Tokyo: 22°C, Partly cloudy, 60% humidity, Wind: 8 km/h E
Weather data not available for 'Mars'. Available cities: London, Tokyo, New York, Sydney, Paris, Mumbai, San Francisco


### 3.2 Calculator Tool Example

In [13]:
def calculate(expression: str) -> str:
    """Safely evaluate mathematical expressions.
    
    Args:
        expression: Mathematical expression to evaluate
        
    Returns:
        Calculation result or error message
    """
    try:
        # Whitelist allowed characters for safety
        allowed_chars = set('0123456789+-*/()., ')
        
        # Check for potentially dangerous operations
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic math operations are allowed (+, -, *, /, parentheses, and numbers)"
        
        # Additional safety checks
        dangerous_terms = ['import', 'exec', 'eval', '__', 'open', 'file']
        if any(term in expression.lower() for term in dangerous_terms):
            return "Error: Potentially dangerous operation detected"
        
        # Evaluate the expression
        result = eval(expression)
        
        # Format the result nicely
        if isinstance(result, float):
            # Round to reasonable precision
            if result.is_integer():
                result = int(result)
            else:
                result = round(result, 6)
        
        return f"Result: {result}"
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except SyntaxError:
        return "Error: Invalid mathematical expression syntax"
    except Exception as e:
        return f"Error: {str(e)}"

# Define the tool specification
calculator_tool = {
    "type": "function", 
    "function": {
        "name": "calculate",
        "description": "Perform mathematical calculations with basic arithmetic operations",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Mathematical expression to evaluate (e.g., '2+2', '15*0.20', '(100-25)/3')"
                }
            },
            "required": ["expression"]
        }
    }
}

# Test the function directly
print("🧪 Testing calculator function directly:")
print(calculate("2 + 2"))
print(calculate("15 * 0.20"))
print(calculate("(100 - 25) / 3"))
print(calculate("10 / 0"))  # Test error handling
print(calculate("import os"))  # Test security

🧪 Testing calculator function directly:
Result: 4
Result: 3
Result: 25
Error: Division by zero
Error: Only basic math operations are allowed (+, -, *, /, parentheses, and numbers)


### 3.3 Knowledge Search Tool Example

In [14]:
def search_knowledge(query: str) -> str:
    """Search a simulated knowledge base for information.
    
    In a real application, this could search a vector database,
    call a search API, or query a documentation system.
    
    Args:
        query: Search query or topic to look up
        
    Returns:
        Information about the topic or a "not found" message
    """
    # Simulated knowledge base
    knowledge_base = {
        "python": "Python is a high-level programming language known for its simplicity and readability. Created by Guido van Rossum in 1991, it's widely used for web development, data science, AI, and automation.",
        
        "machine learning": "Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions.",
        
        "neural networks": "Neural networks are computing systems inspired by biological neural networks. They consist of interconnected nodes (neurons) that process information and can learn complex patterns through training on data.",
        
        "api": "API stands for Application Programming Interface. It's a set of protocols, routines, and tools that allow different software applications to communicate with each other.",
        
        "docker": "Docker is a containerization platform that allows developers to package applications and their dependencies into lightweight, portable containers that can run consistently across different environments.",
        
        "kubernetes": "Kubernetes is an open-source container orchestration platform that automates the deployment, scaling, and management of containerized applications across clusters of hosts.",
        
        "git": "Git is a distributed version control system used to track changes in source code during software development. It allows multiple developers to collaborate on projects efficiently.",
        
        "javascript": "JavaScript is a versatile programming language primarily used for web development. It runs in browsers and enables interactive web pages, but is also used for server-side development with Node.js."
    }
    
    # Normalize query for searching
    query_lower = query.lower().strip()
    
    # Try exact match first
    if query_lower in knowledge_base:
        return f"📚 Information about {query_lower.title()}: {knowledge_base[query_lower]}"
    
    # Try partial matches
    matches = []
    for key, value in knowledge_base.items():
        if query_lower in key or key in query_lower:
            matches.append((key, value))
    
    if matches:
        if len(matches) == 1:
            key, value = matches[0]
            return f"📚 Information about {key.title()}: {value}"
        else:
            result = f"📚 Found {len(matches)} topics related to '{query}': "
            for key, value in matches:
                result += f"\n\n{key.title()}: {value}"
            return result
    
    # No matches found
    available_topics = ", ".join([topic.title() for topic in knowledge_base.keys()])
    return f"❌ No information found for '{query}'. Available topics: {available_topics}"

# Define the tool specification  
search_tool = {
    "type": "function",
    "function": {
        "name": "search_knowledge",
        "description": "Search for information in the knowledge base about programming, technology, and software development topics",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string", 
                    "description": "Search query or topic to look up (e.g., 'python', 'machine learning', 'api')"
                }
            },
            "required": ["query"]
        }
    }
}

# Test the function directly
print("🧪 Testing knowledge search function directly:")
print(search_knowledge("python"))
print("\n" + search_knowledge("machine"))  # Partial match
print("\n" + search_knowledge("blockchain"))  # Not found

🧪 Testing knowledge search function directly:
📚 Information about Python: Python is a high-level programming language known for its simplicity and readability. Created by Guido van Rossum in 1991, it's widely used for web development, data science, AI, and automation.

📚 Information about Machine Learning: Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions.

❌ No information found for 'blockchain'. Available topics: Python, Machine Learning, Neural Networks, Api, Docker, Kubernetes, Git, Javascript


## 4. Function Calling Integration

Now that we have our tools defined, let's integrate them with the LLM API. This involves:

1. Making API calls with tool definitions
2. Parsing tool call responses
3. Executing the requested functions
4. Feeding results back to the conversation

### 4.1 Making API Calls with Tools

In [10]:
def call_with_tools(prompt, tools, model="google/gemini-2.5-flash-preview", **kwargs):
    """Make an API call with tool support using Gemini.
    
    Args:
        prompt: The user's prompt or conversation messages
        tools: List of tool definitions in tools format
        model: Model to use (default: Gemini 2.5 Flash Preview)
        **kwargs: Additional parameters for the API call
        
    Returns:
        API response dictionary
    """
    # Convert tools format to functions format for call_openrouter
    # The Day 2 api_utils expects 'functions' parameter, not 'tools'
    functions = []
    for tool in tools:
        if tool.get("type") == "function":
            functions.append(tool["function"])
        else:
            # If it's already in function format, use as-is
            functions.append(tool)
    
    # Create a system prompt that encourages tool use with Gemini
    system_prompt = "You are a helpful assistant with access to tools. When the user asks for information that can be obtained through tools, you should use the appropriate tool rather than providing a general response."
    
    # Prepare the request with functions
    response = call_openrouter(
        prompt=prompt,
        model=model,
        system_prompt=system_prompt,
        functions=functions if functions else None,
        temperature=kwargs.get('temperature', 0.3),  # Lower temperature for more reliable tool use
        max_tokens=kwargs.get('max_tokens', 300)
    )
    return response

# Create our tool registry - maps function names to actual functions
tool_functions = {
    "get_weather": get_weather,
    "calculate": calculate, 
    "search_knowledge": search_knowledge
}

# List of all our tools
all_tools = [weather_tool, calculator_tool, search_tool]

print("🔧 Tool registry created with functions:")
for name, func in tool_functions.items():
    print(f"  • {name}: {func.__doc__.split('.')[0] if func.__doc__ else 'No description'}")
    
print(f"\n📋 Available tools: {len(all_tools)} tools ready for Gemini 2.5 Flash model")

🔧 Tool registry created with functions:
  • get_weather: Get current weather for a location (simulated with realistic data)
  • calculate: Safely evaluate mathematical expressions
  • search_knowledge: Search a simulated knowledge base for information

📋 Available tools: 3 tools ready for Gemini 2.5 Flash model


In [11]:
import json  # Add missing import

def extract_and_execute_tool_call(response, tool_functions):
    """Extract and execute tool calls from API response.
    
    Args:
        response: API response dictionary from call_openrouter
        tool_functions: Dictionary mapping function names to actual functions
        
    Returns:
        tuple: (tool_call_info, execution_result) or (None, None) if no tool call
    """
    if not response.get("success", False):
        return None, f"API Error: {response.get('error', 'Unknown error')}"
    
    try:
        # Extract the message from the response
        message = response["response"]["choices"][0]["message"]
        
        # Check for tool_calls (new format) first
        tool_calls = message.get("tool_calls", [])
        if tool_calls and len(tool_calls) > 0:
            tool_call = tool_calls[0]  # Get the first tool call
            function_info = {
                "success": True,
                "tool_id": tool_call.get("id"),
                "function_name": tool_call.get("function", {}).get("name"),
                "arguments": json.loads(tool_call.get("function", {}).get("arguments", "{}"))
            }
            print(f"✅ Found tool call: {function_info['function_name']} with args: {function_info['arguments']}")
        else:
            # Fall back to the original extract_function_call for legacy formats
            function_info = extract_function_call(response)
            if not function_info.get("success", False):
                return None, None
        
        function_name = function_info.get("function_name")
        arguments = function_info.get("arguments", {})
        
        if function_name not in tool_functions:
            return function_info, f"Error: Function '{function_name}' not found in tool registry"
        
        # Execute the function with the extracted arguments
        try:
            func = tool_functions[function_name]
            
            # Handle different argument patterns
            if isinstance(arguments, dict):
                if len(arguments) == 1 and function_name in ["get_weather", "search_knowledge", "calculate"]:
                    # Single argument functions - pass the first value
                    arg_value = list(arguments.values())[0]
                    result = func(arg_value)
                elif function_name == "convert_units" and len(arguments) == 3:
                    # Multi-argument function - pass all arguments
                    result = func(**arguments)
                else:
                    # Try to pass all arguments as keyword arguments
                    result = func(**arguments)
            else:
                # Fallback for non-dict arguments
                result = func(arguments)
            
            return function_info, result
            
        except Exception as e:
            return function_info, f"Error executing {function_name}: {str(e)}"
            
    except Exception as e:
        return None, f"Error processing tool call: {str(e)}"

# Test the extraction function with a simple example
def demo_tool_extraction():
    """Demonstrate tool call extraction with a live example."""
    print("🧪 Testing tool call extraction...")
    
    # Try a more explicit prompt
    test_prompt = "Use the get_weather function to get the current weather in Tokyo. You must call the function, do not provide general weather information."
    
    test_response = call_with_tools(test_prompt, all_tools)
    
    if test_response.get("success"):
        tool_call, result = extract_and_execute_tool_call(test_response, tool_functions)
        if tool_call:
            print("\n🎯 Tool call processed successfully!")
            
            # Display results with proper formatting
            tool_info = f"""
**Function Called:** `{tool_call.get('function_name', 'Unknown')}`  
**Arguments:** `{tool_call.get('arguments', {})}`  
**Result:** {result}
            """
            display_markdown(tool_info, "Tool Execution Results")
            
        else:
            regular_response = extract_text_response(test_response)
            print(f"\n❌ No tool call detected. Regular response:")
            display_llm_response(regular_response, title="Regular Response (No Tool Call)")
    else:
        display_error(f"API call failed: {test_response.get('error')}")

demo_tool_extraction()

🧪 Testing tool call extraction...
✅ Found tool call: get_weather with args: {'location': 'Tokyo'}

🎯 Tool call processed successfully!


### Tool Execution Results


**Function Called:** `get_weather`  
**Arguments:** `{'location': 'Tokyo'}`  
**Result:** Current weather in Tokyo: 22°C, Partly cloudy, 60% humidity, Wind: 8 km/h E
            

### 4.3 Complete Working Example: Multi-Tool Agent

Now let's put it all together to create a complete agent that can use multiple tools to answer complex questions. We'll use a simple, reliable approach that avoids complex conversation state management.

In [12]:
# Working Multi-Tool Agent Solution with Gemini
def simple_multi_tool_approach(user_query):
    """Simple approach: execute each tool individually, then combine results."""
    print(f"🤖 Simple multi-tool approach with Gemini: '{user_query}'")
    
    results = []
    gemini_model = "google/gemini-2.5-flash-preview"
    
    # Step 1: Check if weather is needed and get it
    if "weather" in user_query.lower() or any(city in user_query.lower() for city in ["london", "tokyo", "paris", "new york", "sydney"]):
        print("\n🌤️ Getting weather information...")
        weather_conversation = [
            {"role": "system", "content": "You are a weather assistant. Use the get_weather tool to get current weather information. You must use the tool, not provide general information."},
            {"role": "user", "content": user_query}
        ]
        
        weather_response = call_openrouter(
            prompt=weather_conversation,
            model=gemini_model,
            functions=[weather_tool["function"]],
            temperature=0.2,
            max_tokens=300
        )
        
        if weather_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(weather_response, {"get_weather": get_weather})
            if tool_call:
                results.append(("Weather", result))
                print(f"✅ Weather: {result}")
    
    # Step 2: Check if calculation is needed and do it
    if any(word in user_query.lower() for word in ["calculate", "tip", "percent", "%", "math", "+", "-", "*", "/"]):
        print("\n🧮 Performing calculation...")
        calc_conversation = [
            {"role": "system", "content": "You are a calculation assistant. Use the calculate tool to perform mathematical calculations. You must use the tool for any mathematical operations."},
            {"role": "user", "content": user_query}
        ]
        
        calc_response = call_openrouter(
            prompt=calc_conversation,
            model=gemini_model,
            functions=[calculator_tool["function"]],
            temperature=0.2,
            max_tokens=300
        )
        
        if calc_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(calc_response, {"calculate": calculate})
            if tool_call:
                results.append(("Calculation", result))
                print(f"✅ Calculation: {result}")
    
    # Step 3: Check if knowledge search is needed
    if any(word in user_query.lower() for word in ["search", "find", "lookup", "information", "about", "what is", "tell me about"]):
        print("\n🔍 Searching knowledge base...")
        search_conversation = [
            {"role": "system", "content": "You are a knowledge assistant. Use the search_knowledge tool to find information. You must use the tool to search for information."},
            {"role": "user", "content": user_query}
        ]
        
        search_response = call_openrouter(
            prompt=search_conversation,
            model=gemini_model,
            functions=[search_tool["function"]],
            temperature=0.2,
            max_tokens=300
        )
        
        if search_response.get("success"):
            tool_call, result = extract_and_execute_tool_call(search_response, {"search_knowledge": search_knowledge})
            if tool_call:
                results.append(("Knowledge", result))
                print(f"✅ Knowledge: {result}")
    
    # Step 4: Combine results into final response
    if results:
        print(f"\n📝 Combining {len(results)} results...")
        
        results_text = "Based on your request:\n\n"
        for category, result in results:
            results_text += f"**{category}**: {result}\n\n"
        
        final_prompt = f"""User asked: "{user_query}"

{results_text}

Please provide a helpful and comprehensive response to the user based on this information."""
        
        final_response = call_openrouter(
            prompt=final_prompt,
            model=gemini_model,
            temperature=0.7,
            max_tokens=300
        )
        
        if final_response.get("success"):
            return extract_text_response(final_response)
        else:
            return f"Error in final response: {final_response.get('error')}"
    else:
        # No tools were used, just answer directly
        print("📝 No tools needed, providing direct response...")
        direct_response = call_openrouter(
            prompt=user_query,
            model=gemini_model,
            temperature=0.7,
            max_tokens=300
        )
        
        if direct_response.get("success"):
            return extract_text_response(direct_response)
        else:
            return f"Error in direct response: {direct_response.get('error')}"

# Test the simple multi-tool approach with Gemini
print("🚀 TESTING SIMPLE MULTI-TOOL APPROACH WITH GEMINI 2.5 FLASH")
print("=" * 60)

# Test 1: Complex multi-tool query
print("\n**TEST 1: Multi-tool query**")
complex_result = simple_multi_tool_approach("I'm planning a trip to London. Tell me the weather and calculate 20% tip on £150.")
display_llm_response(complex_result, model="Gemini 2.5 Flash Multi-tool Agent", title="London Trip Planning Result")

print("\n" + "=" * 60)

# Test 2: Single tool query
print("\n**TEST 2: Weather query**")
weather_result = simple_multi_tool_approach("What's the weather in Tokyo?")
display_llm_response(weather_result, model="Gemini 2.5 Flash Multi-tool Agent", title="Tokyo Weather Result")

print("\n" + "=" * 60)

# Test 3: Calculation query
print("\n**TEST 3: Calculation query**")
calc_result = simple_multi_tool_approach("Calculate 25 * 4 + 100")
display_llm_response(calc_result, model="Gemini 2.5 Flash Multi-tool Agent", title="Mathematical Calculation Result")

print("\n" + "=" * 60)

# Test 4: Knowledge query
print("\n**TEST 4: Knowledge query**")
knowledge_result = simple_multi_tool_approach("Tell me about Python programming language")
display_llm_response(knowledge_result, model="Gemini 2.5 Flash Multi-tool Agent", title="Python Knowledge Lookup Result")

🚀 TESTING SIMPLE MULTI-TOOL APPROACH WITH GEMINI 2.5 FLASH

**TEST 1: Multi-tool query**
🤖 Simple multi-tool approach with Gemini: 'I'm planning a trip to London. Tell me the weather and calculate 20% tip on £150.'

🌤️ Getting weather information...
✅ Found tool call: get_weather with args: {'location': 'London'}
✅ Weather: Current weather in London: 15°C, Light rain, 80% humidity, Wind: 12 km/h SW

🧮 Performing calculation...
✅ Found tool call: calculate with args: {'expression': '150 * 0.20'}
✅ Calculation: Result: 30

📝 Combining 2 results...


Okay, here's a helpful response for your trip planning to London:

**London Weather & Tip Calculation:**

Here's the information you requested for your trip to London:

* **Weather:** Currently in London, it's **15°C** with **light rain**. The humidity is **80%**, and there's a wind coming from the **southwest** at **12 km/h**.

* **Tip Calculation:** A 20% tip on £150 is **£30**.

**Tips for Your London Trip based on this information:**

* **Be prepared for rain:** Given the light rain, it's a good idea to pack a waterproof jacket or umbrella for your trip.
* **Layer up:** 15°C is cool, so bring layers of clothing so you can adjust to the temperature throughout the day.
* **Keep an eye on the forecast:** While this is the current weather, London weather can change quickly. Check the forecast closer to your travel dates and during your trip for the most up-to-date information.
* **Tipping in London:** While tipping is appreciated in London, it's not as ingrained as in some other countries. For restaurant service, a 10-15% tip is more common, especially if a service charge hasn't already been added to the bill. However, 20% is certainly generous and will be



**TEST 2: Weather query**
🤖 Simple multi-tool approach with Gemini: 'What's the weather in Tokyo?'

🌤️ Getting weather information...
✅ Found tool call: get_weather with args: {'location': 'Tokyo'}
✅ Weather: Current weather in Tokyo: 22°C, Partly cloudy, 60% humidity, Wind: 8 km/h E

📝 Combining 1 results...


Okay, here's a helpful and comprehensive response about the weather in Tokyo based on the information you provided:

**"The current weather in Tokyo is 22°C and it's partly cloudy. The humidity is 60%, and there's a light breeze coming from the east at 8 km/h."**

Here are some additional ways you could frame the response, depending on the context of the conversation:

**For a quick and concise answer:**

> "It's currently 22°C and partly cloudy in Tokyo. There's 60% humidity and a light easterly wind at 8 km/h."

**For a slightly more descriptive answer:**

> "Right now in Tokyo, it's a pleasant 22°C with some clouds in the sky. The air feels a bit humid at 60%, and there's a gentle 8 km/h wind blowing from the east."

**If you wanted to give a little more context (though not strictly necessary based *only* on the provided info):**

> "The weather in Tokyo is currently 22°C and partly cloudy. With 60% humidity and a light 8 km/h easterly wind, it seems like a comfortable day."

**Key elements of a helpful and comprehensive response include:**

* **Directly answering the question:** Starting with the temperature and the general conditions (partly



**TEST 3: Calculation query**
🤖 Simple multi-tool approach with Gemini: 'Calculate 25 * 4 + 100'

🧮 Performing calculation...
✅ Found tool call: calculate with args: {'expression': '25 * 4 + 100'}
✅ Calculation: Result: 200

📝 Combining 1 results...


Based on your request to calculate **25 * 4 + 100**, here's the breakdown and the result:

**Calculation:**

We follow the order of operations (PEMDAS/BODMAS), which means we perform multiplication before addition:

1. **Multiplication:** 25 * 4 = 100
2. **Addition:** 100 + 100 = 200

**Result:**

The result of 25 * 4 + 100 is **200**.

Let me know if you have any other calculations you'd like me to perform!



**TEST 4: Knowledge query**
🤖 Simple multi-tool approach with Gemini: 'Tell me about Python programming language'

🔍 Searching knowledge base...
✅ Found tool call: search_knowledge with args: {'query': 'Python programming language'}
✅ Knowledge: 📚 Information about Python: Python is a high-level programming language known for its simplicity and readability. Created by Guido van Rossum in 1991, it's widely used for web development, data science, AI, and automation.

📝 Combining 1 results...


Okay, let's talk about Python!

Based on the information you provided and general knowledge about Python, here's a comprehensive overview:

**Python: A Powerful and Versatile Programming Language**

Python is a **high-level programming language** that was created by Guido van Rossum and first released in 1991. It's renowned for its **simplicity and readability**, which makes it a great language for beginners to learn.

Here's a breakdown of what makes Python so popular and widely used:

* **Readability and Simplicity:** Python's syntax is designed to be intuitive and easy to understand, resembling natural language more than many other programming languages. This reduces the cognitive load and makes it easier to write and maintain code.
* **Versatility and Wide Applications:** Python is a truly versatile language used across a vast range of domains. Some of its key areas of application include:
    * **Web Development:** Frameworks like Django and Flask are incredibly popular for building robust and scalable web applications.
    * **Data Science and Analytics:** Python is a cornerstone of the data science world with powerful libraries like NumPy, Pandas, and Scikit-learn for data manipulation, analysis, and machine learning.
    * **Artificial Intelligence (AI) and Machine Learning (ML):** Python is the go-to language for AI and ML development, thanks to libraries like TensorFlow and PyTorch.
    * **Automation and Scripting

## 5. Key Takeaways and Best Practices

### 5.1 What We've Learned

1. **Tools extend LLM capabilities** beyond their training data
2. **Proper tool definition** is crucial for reliable function calling
3. **Error handling** in tool functions prevents agent failures
4. **Simple architectures** are often more reliable than complex ones
5. **Sequential tool execution** avoids conversation state complexity

### 5.2 Best Practices

1. **Clear tool descriptions**: Help the LLM understand when to use each tool
2. **Input validation**: Always validate and sanitize tool inputs
3. **Error handling**: Gracefully handle tool execution failures
4. **Security**: Never allow dangerous operations in tools
5. **Performance**: Keep tool execution fast and reliable
6. **Architecture**: Choose simple, maintainable patterns over complex ones

### 5.3 Common Patterns

- **Information retrieval**: Weather, search, lookup tools
- **Computation**: Calculator, data analysis, conversion tools  
- **External actions**: Email, booking, file operations
- **Validation**: Fact-checking, data verification tools

### 5.4 Production Considerations

- **Rate limiting**: Respect API limits and implement backoff strategies
- **Caching**: Cache tool results when appropriate
- **Monitoring**: Log tool usage and performance metrics
- **Fallbacks**: Have backup strategies when tools fail

## 6. Next Steps

In the next notebook, we'll explore:

- **Advanced agent architectures**: ReAct pattern, planning agents, and tool chains
- **Error recovery**: Handling tool failures gracefully
- **Performance optimization**: Caching, parallel execution, and efficiency
- **Real-world integration**: Connecting to actual APIs and services
- **Agent evaluation**: Testing and measuring agent performance

You now have the foundation to build sophisticated agents that can interact with the real world through tools!

## 7. Practice Exercise

Try creating your own tool! Here's a template:

In [ ]:
# Exercise: Create a unit converter tool
def convert_units(value: float, from_unit: str, to_unit: str) -> str:
    """Convert between different units.
    
    Args:
        value: The numeric value to convert
        from_unit: The source unit (e.g., 'celsius', 'fahrenheit', 'meters', 'feet')
        to_unit: The target unit
        
    Returns:
        Conversion result or error message
    """
    # Unit conversion mappings
    conversions = {
        ('celsius', 'fahrenheit'): lambda x: x * 9/5 + 32,
        ('fahrenheit', 'celsius'): lambda x: (x - 32) * 5/9,
        ('meters', 'feet'): lambda x: x * 3.28084,
        ('feet', 'meters'): lambda x: x / 3.28084,
        ('kilometers', 'miles'): lambda x: x * 0.621371,
        ('miles', 'kilometers'): lambda x: x / 0.621371,
        ('kilograms', 'pounds'): lambda x: x * 2.20462,
        ('pounds', 'kilograms'): lambda x: x / 2.20462,
    }
    
    key = (from_unit.lower(), to_unit.lower())
    if key in conversions:
        result = conversions[key](value)
        return f"{value} {from_unit} = {result:.2f} {to_unit}"
    else:
        available = [f"{f} → {t}" for f, t in conversions.keys()]
        return f"Conversion from {from_unit} to {to_unit} not supported. Available: {', '.join(available[:3])}..."

# Define the tool specification for your converter
converter_tool = {
    "type": "function",
    "function": {
        "name": "convert_units",
        "description": "Convert between different units of measurement (temperature, distance, weight)",
        "parameters": {
            "type": "object",
            "properties": {
                "value": {
                    "type": "number",
                    "description": "The numeric value to convert"
                },
                "from_unit": {
                    "type": "string", 
                    "description": "Source unit (celsius, fahrenheit, meters, feet, kilometers, miles, kilograms, pounds)"
                },
                "to_unit": {
                    "type": "string",
                    "description": "Target unit for conversion"
                }
            },
            "required": ["value", "from_unit", "to_unit"]
        }
    }
}

# Test your tool with markdown display
print("🧪 Testing unit converter:")

test_cases = [
    (100, "celsius", "fahrenheit"),
    (10, "meters", "feet"),
    (5, "kilometers", "miles")
]

for value, from_unit, to_unit in test_cases:
    result = convert_units(value, from_unit, to_unit)
    display_markdown(f"**{value} {from_unit} → {to_unit}:** {result}")

# Challenge: Integrate this tool into the simple_multi_tool_approach function!
challenge_text = """
### 🎯 **Challenge**

Try integrating your unit converter tool into the `simple_multi_tool_approach()` function:

1. **Add unit conversion detection** - Check for keywords like "convert", "temperature", "distance"
2. **Create the tool conversation** - Set up a system prompt for unit conversion
3. **Execute and integrate** - Add the conversion result to the final response
4. **Test it** - Try a query like "Convert 100 celsius to fahrenheit and tell me the weather in Paris"

This exercise demonstrates how to extend the multi-tool agent with new capabilities!
"""

display_markdown(challenge_text, "Practice Challenge")